## Fixing Student Info

In [3]:
import pandas as pd
import numpy as np


In [4]:
df = pd.read_excel('data/address_data.xlsx')
df.head()

,Unique ID,Gardian Name,Email,Address,Scholar Name,Scholar Grade,Scholar 1 Enrollment Status
0,7356714015,London Cameron,rasca@outlook.com,2856 Rhonda Lane Denver CO 80207,Atticus Jackson,K,Application Submitted
1,3233233322,Marley Allen,Marley.Allen@gmail.com,1638 E Washington St Denver CO 80219,Carlo Huang,Second,Currently Enrolled
2,4344344433,Yair Harrington,Yair.Harrington@gmail.com,628 n Lafayette st. Denver CO 80219,Mitchell Castillo,Second,Currently Enrolled
3,5467891234,Ashanti Huynh,Ashanti.Huynh@gmail.com,58 S 2nd street Denver CO 80219,Melina Ward,Second,Currently Enrolled
4,7899877799,Ashanti Sellers,Ashanti.Sellers@gmail.com,58 S 2nd Street Denver CO 80219,Braden McClain,Second,Currently Enrolled


## Name Columns

Change the Guardian Name and Scholar Name Columns to First and Last name columns for both Scholar and Guardian

In [5]:
def first_name(name):
    result = name.split()[0]
    return result
def last_name(name):
    result = name.split()[1]
    return result
#add columns and split the names
df['Guardian_First'] = df['Gardian Name'].apply(lambda x: first_name(x))
df['Guardian_last'] = df['Gardian Name'].apply(lambda x: last_name(x))
df['Scholar_First'] = df['Scholar Name'].apply(lambda x: first_name(x))
df['Scholar_Last'] = df['Scholar Name'].apply(lambda x: last_name(x))
#drop original columns
df = df.drop(['Gardian Name','Scholar Name'], axis=1)

df.head()

,Unique ID,Email,Address,Scholar Grade,Scholar 1 Enrollment Status,Guardian_First,Guardian_last,Scholar_First,Scholar_Last
0,7356714015,rasca@outlook.com,2856 Rhonda Lane Denver CO 80207,K,Application Submitted,London,Cameron,Atticus,Jackson
1,3233233322,Marley.Allen@gmail.com,1638 E Washington St Denver CO 80219,Second,Currently Enrolled,Marley,Allen,Carlo,Huang
2,4344344433,Yair.Harrington@gmail.com,628 n Lafayette st. Denver CO 80219,Second,Currently Enrolled,Yair,Harrington,Mitchell,Castillo
3,5467891234,Ashanti.Huynh@gmail.com,58 S 2nd street Denver CO 80219,Second,Currently Enrolled,Ashanti,Huynh,Melina,Ward
4,7899877799,Ashanti.Sellers@gmail.com,58 S 2nd Street Denver CO 80219,Second,Currently Enrolled,Ashanti,Sellers,Braden,McClain


## Fix the Address Column

Goind to try using usaaddress and/or scourgify to create my address columns.

In [6]:
import usaddress

In [7]:
# Parse and label an address
address = "58 S 2nd Street Denver CO 80219"
parsed_address = usaddress.parse(address)

# Print the parsed address
print(parsed_address)

[('58', 'AddressNumber'), ('S', 'StreetNamePreDirectional'), ('2nd', 'StreetName'), ('Street', 'StreetNamePostType'), ('Denver', 'PlaceName'), ('CO', 'StateName'), ('80219', 'ZipCode')]


This could work but let's look at scourgify which is built on top of usaddress and will do some of lifting for us.

In [8]:
#scourgify https://github.com/GreenBuildingRegistry/usaddress-scourgify
from scourgify import normalize_address_record

record = normalize_address_record('58 s 2nd Street Denver CO 80219')

record

{'address_line_1': '58 S 2ND ST',
 'address_line_2': None,
 'city': 'DENVER',
 'state': 'CO',
 'postal_code': '80219'}

In [9]:
record['address_line_1']

'58 S 2ND ST'

In [13]:
print(record['address_line_2'])

None


This looks like a good starting point. We can now build a function convert the address column into it's individual parts.

In [17]:
## function 
def address_separator(a):
    record = normalize_address_record(a)
    #check second address line
    add2 = ""
    if record['address_line_2']:
        add2=record['address_line_2']
    #create parts of the address
    street = record['address_line_1'] + add2
    city = record['city']
    state = record['state']
    zip = record['postal_code']
    return [street, city, state, zip]

address_separator('58 s 2nd Street Denver CO 80219')

['58 S 2ND ST', 'DENVER', 'CO', '80219']

The first step will be to temporarily create a column that has a list of the parts.

In [20]:
#apply this function to each row creating address part column
df['add_parts'] = df['Address'].apply(lambda row: address_separator(row))

df.head()

,Unique ID,Email,Address,Scholar Grade,Scholar 1 Enrollment Status,Guardian_First,Guardian_last,Scholar_First,Scholar_Last,add_parts
0,7356714015,rasca@outlook.com,2856 Rhonda Lane Denver CO 80207,K,Application Submitted,London,Cameron,Atticus,Jackson,"[2856 RHONDA LN, DENVER, CO, 80207]"
1,3233233322,Marley.Allen@gmail.com,1638 E Washington St Denver CO 80219,Second,Currently Enrolled,Marley,Allen,Carlo,Huang,"[1638 E WASHINGTON ST, DENVER, CO, 80219]"
2,4344344433,Yair.Harrington@gmail.com,628 n Lafayette st. Denver CO 80219,Second,Currently Enrolled,Yair,Harrington,Mitchell,Castillo,"[628 N LAFAYETTE ST, DENVER, CO, 80219]"
3,5467891234,Ashanti.Huynh@gmail.com,58 S 2nd street Denver CO 80219,Second,Currently Enrolled,Ashanti,Huynh,Melina,Ward,"[58 S 2ND ST, DENVER, CO, 80219]"
4,7899877799,Ashanti.Sellers@gmail.com,58 S 2nd Street Denver CO 80219,Second,Currently Enrolled,Ashanti,Sellers,Braden,McClain,"[58 S 2ND ST, DENVER, CO, 80219]"


create columns from the add_parts

In [24]:
# assign values to columns
edf['street'] = df['add_parts'].apply(lambda x: x[0])
df['city'] = df['add_parts'].apply(lambda x: x[1])
df['state']= df['add_parts'].apply(lambda x: x[2])
df['zip'] = df['add_parts'].apply(lambda x: x[3])
df.head()

,Unique ID,Email,Address,Scholar Grade,Scholar 1 Enrollment Status,Guardian_First,Guardian_last,Scholar_First,Scholar_Last,add_parts,street,city,state,zip
0,7356714015,rasca@outlook.com,2856 Rhonda Lane Denver CO 80207,K,Application Submitted,London,Cameron,Atticus,Jackson,"[2856 RHONDA LN, DENVER, CO, 80207]",2856 RHONDA LN,DENVER,CO,80207
1,3233233322,Marley.Allen@gmail.com,1638 E Washington St Denver CO 80219,Second,Currently Enrolled,Marley,Allen,Carlo,Huang,"[1638 E WASHINGTON ST, DENVER, CO, 80219]",1638 E WASHINGTON ST,DENVER,CO,80219
2,4344344433,Yair.Harrington@gmail.com,628 n Lafayette st. Denver CO 80219,Second,Currently Enrolled,Yair,Harrington,Mitchell,Castillo,"[628 N LAFAYETTE ST, DENVER, CO, 80219]",628 N LAFAYETTE ST,DENVER,CO,80219
3,5467891234,Ashanti.Huynh@gmail.com,58 S 2nd street Denver CO 80219,Second,Currently Enrolled,Ashanti,Huynh,Melina,Ward,"[58 S 2ND ST, DENVER, CO, 80219]",58 S 2ND ST,DENVER,CO,80219
4,7899877799,Ashanti.Sellers@gmail.com,58 S 2nd Street Denver CO 80219,Second,Currently Enrolled,Ashanti,Sellers,Braden,McClain,"[58 S 2ND ST, DENVER, CO, 80219]",58 S 2ND ST,DENVER,CO,80219


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 14 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Unique ID                    189 non-null    object
 1   Email                        185 non-null    object
 2   Address                      189 non-null    object
 3   Scholar Grade                186 non-null    object
 4   Scholar 1 Enrollment Status  189 non-null    object
 5   Guardian_First               189 non-null    object
 6   Guardian_last                189 non-null    object
 7   Scholar_First                189 non-null    object
 8   Scholar_Last                 189 non-null    object
 9   add_parts                    189 non-null    object
 10  street                       189 non-null    object
 11  city                         189 non-null    object
 12  state                        189 non-null    object
 13  zip                          189 no

In [28]:
df = df.drop(['Address','add_parts'],axis=1)
df.head()

,Unique ID,Email,Scholar Grade,Scholar 1 Enrollment Status,Guardian_First,Guardian_last,Scholar_First,Scholar_Last,street,city,state,zip
0,7356714015,rasca@outlook.com,K,Application Submitted,London,Cameron,Atticus,Jackson,2856 RHONDA LN,DENVER,CO,80207
1,3233233322,Marley.Allen@gmail.com,Second,Currently Enrolled,Marley,Allen,Carlo,Huang,1638 E WASHINGTON ST,DENVER,CO,80219
2,4344344433,Yair.Harrington@gmail.com,Second,Currently Enrolled,Yair,Harrington,Mitchell,Castillo,628 N LAFAYETTE ST,DENVER,CO,80219
3,5467891234,Ashanti.Huynh@gmail.com,Second,Currently Enrolled,Ashanti,Huynh,Melina,Ward,58 S 2ND ST,DENVER,CO,80219
4,7899877799,Ashanti.Sellers@gmail.com,Second,Currently Enrolled,Ashanti,Sellers,Braden,McClain,58 S 2ND ST,DENVER,CO,80219


## Fix Scholar Grade

The goal for this column is to get uniform grade labels.  The desired options are 
- Kindergarten
- 1st 
- 2nd
- PreK 3

In [34]:
#check the unique_values:
un_list = df['Scholar Grade'].unique().tolist()
un_list

['K', 'Second', '1st', '2nd', 'Kindergarten', 'First', 'PreK 3', nan]

In [36]:
#create a dictionary matching each unique value to the desired string 
#these are only the ones that we need to change
grade_dict = {'K':'Kindergarten','Second':'2nd','First':'1st'}
grade_dict

{'K': 'Kindergarten', 'Second': '2nd', 'First': '1st'}

In [38]:
# creating a new column for demonstrative purposes.  
df['Scholar_Grade'] = df['Scholar Grade'].map(grade_dict).fillna(df['Scholar Grade'])
df.head()

,Unique ID,Email,Scholar Grade,Scholar 1 Enrollment Status,Guardian_First,Guardian_last,Scholar_First,Scholar_Last,street,city,state,zip,Scholar_Grade
0,7356714015,rasca@outlook.com,K,Application Submitted,London,Cameron,Atticus,Jackson,2856 RHONDA LN,DENVER,CO,80207,Kindergarten
1,3233233322,Marley.Allen@gmail.com,Second,Currently Enrolled,Marley,Allen,Carlo,Huang,1638 E WASHINGTON ST,DENVER,CO,80219,2nd
2,4344344433,Yair.Harrington@gmail.com,Second,Currently Enrolled,Yair,Harrington,Mitchell,Castillo,628 N LAFAYETTE ST,DENVER,CO,80219,2nd
3,5467891234,Ashanti.Huynh@gmail.com,Second,Currently Enrolled,Ashanti,Huynh,Melina,Ward,58 S 2ND ST,DENVER,CO,80219,2nd
4,7899877799,Ashanti.Sellers@gmail.com,Second,Currently Enrolled,Ashanti,Sellers,Braden,McClain,58 S 2ND ST,DENVER,CO,80219,2nd


### Drop Column for sholar grade
We can now drop the original columns 'Scholar Grade' 

In [41]:
df = df.drop(['Scholar Grade'],axis=1)
df.head()

,Unique ID,Email,Scholar 1 Enrollment Status,Guardian_First,Guardian_last,Scholar_First,Scholar_Last,street,city,state,zip,Scholar_Grade
0,7356714015,rasca@outlook.com,Application Submitted,London,Cameron,Atticus,Jackson,2856 RHONDA LN,DENVER,CO,80207,Kindergarten
1,3233233322,Marley.Allen@gmail.com,Currently Enrolled,Marley,Allen,Carlo,Huang,1638 E WASHINGTON ST,DENVER,CO,80219,2nd
2,4344344433,Yair.Harrington@gmail.com,Currently Enrolled,Yair,Harrington,Mitchell,Castillo,628 N LAFAYETTE ST,DENVER,CO,80219,2nd
3,5467891234,Ashanti.Huynh@gmail.com,Currently Enrolled,Ashanti,Huynh,Melina,Ward,58 S 2ND ST,DENVER,CO,80219,2nd
4,7899877799,Ashanti.Sellers@gmail.com,Currently Enrolled,Ashanti,Sellers,Braden,McClain,58 S 2ND ST,DENVER,CO,80219,2nd


## Reorder Columns
We will reorder the columns back to their original spots

In [43]:
cols = df.columns.tolist()
cols

['Unique ID',
 'Email',
 'Scholar 1 Enrollment Status',
 'Guardian_First',
 'Guardian_last',
 'Scholar_First',
 'Scholar_Last',
 'street',
 'city',
 'state',
 'zip',
 'Scholar_Grade']

In [45]:
#new column order
correct_order = ['Unique ID','Guardian_First','Guardian_last','Email','street','city','state','zip',
                'Scholar_First','Scholar_Last','Scholar_Grade','Scholar 1 Enrollment Status']
reordered_df = df[correct_order].copy()

In [47]:
#first 5 results
reordered_df.head()

,Unique ID,Guardian_First,Guardian_last,Email,street,city,state,zip,Scholar_First,Scholar_Last,Scholar_Grade,Scholar 1 Enrollment Status
0,7356714015,London,Cameron,rasca@outlook.com,2856 RHONDA LN,DENVER,CO,80207,Atticus,Jackson,Kindergarten,Application Submitted
1,3233233322,Marley,Allen,Marley.Allen@gmail.com,1638 E WASHINGTON ST,DENVER,CO,80219,Carlo,Huang,2nd,Currently Enrolled
2,4344344433,Yair,Harrington,Yair.Harrington@gmail.com,628 N LAFAYETTE ST,DENVER,CO,80219,Mitchell,Castillo,2nd,Currently Enrolled
3,5467891234,Ashanti,Huynh,Ashanti.Huynh@gmail.com,58 S 2ND ST,DENVER,CO,80219,Melina,Ward,2nd,Currently Enrolled
4,7899877799,Ashanti,Sellers,Ashanti.Sellers@gmail.com,58 S 2ND ST,DENVER,CO,80219,Braden,McClain,2nd,Currently Enrolled


In [48]:
reordered_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 189 entries, 0 to 188
Data columns (total 12 columns):
 #   Column                       Non-Null Count  Dtype 
---  ------                       --------------  ----- 
 0   Unique ID                    189 non-null    object
 1   Guardian_First               189 non-null    object
 2   Guardian_last                189 non-null    object
 3   Email                        185 non-null    object
 4   street                       189 non-null    object
 5   city                         189 non-null    object
 6   state                        189 non-null    object
 7   zip                          189 non-null    object
 8   Scholar_First                189 non-null    object
 9   Scholar_Last                 189 non-null    object
 10  Scholar_Grade                186 non-null    object
 11  Scholar 1 Enrollment Status  189 non-null    object
dtypes: object(12)
memory usage: 17.8+ KB


## Export Files


In [50]:
#export to excel
reordered_df.to_excel('data/exported/cleaned_file.xlsx')